In [53]:
pip install pingouin

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from dmba import classificationSummary

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt
import numpy as np 
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [3]:
train_merge_data = pd.read_csv('train_merge_data.csv')
pd.set_option('display.max_columns',None)
train_merge_data.head()

,Unnamed: 0,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,transactions,city,state,type_y,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay
0,0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,D,13,2013,1,0,1,0,True
1,1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,D,13,2013,1,0,1,0,True
2,2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,D,13,2013,1,0,1,0,True
3,3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,D,13,2013,1,0,1,0,True
4,4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,D,13,2013,1,0,1,0,True


In [31]:
train_merge_data['city'].value_counts()

Quito            1018116
Guayaquil         452496
Cuenca            169686
Santo Domingo     169686
Ambato            113124
Machala           113124
Manta             113124
Latacunga         113124
Babahoyo           56562
Playas             56562
Libertad           56562
Ibarra             56562
Cayambe            56562
Puyo               56562
Esmeraldas         56562
Riobamba           56562
Loja               56562
El Carmen          56562
Quevedo            56562
Daule              56562
Guaranda           56562
Salinas            56562
Name: city, dtype: int64

In [4]:
train_merge_data['workday'] = 1 #true: 1 ,false: 0 

train_merge_data.loc[train_merge_data.Weekday > 4, 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Bridge'  , 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Work Day', 'workday'] = 1
train_merge_data.loc[train_merge_data.type_x == 'Transfer', 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == False), 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == True ), 'workday'] = 1

#family 변수 묶어주기

train_merge_data['family'].replace(['BREAD/BAKERY','DAIRY'], 'num1', inplace=True)
train_merge_data['family'].replace(['AUTOMOTIVE','BABY CARE','BEAUTY','BOOKS','CELEBRATION','FROZEN FOODS','GROCERY II','HARDWARE','HOME AND KITCHEN I', 'HOME AND KITCHEN II',
                                   'HOME APPLIANCES','HOME CARE','LADIESWEAR','LAWN AND GARDEN','LINGERIE','LIQUOR,WINE,BEER','MAGAZINES',
                                   'PET SUPPLIES','PLAYERS AND ELECTRONICS','PREPARED FOODS','PRODUCE','SCHOOL AND OFFICE SUPPLIES','SEAFOOD'], 'num2', inplace=True)
train_merge_data['family'].replace(['Tools','PERSONAL CARE','MEATS','POULTRY','DELI','WINE,BEER','LIQUOR', 'EGGS'], 'num3', inplace=True)
train_merge_data['family'].replace(['BEVERAGES'], 'num4', inplace=True)
train_merge_data['family'].replace(['CLEANING'], 'num5', inplace=True)
train_merge_data['family'].replace(['GROCERY I'], 'num6', inplace=True)

# type_y 분산분석

In [15]:
#정규성 확인, 정규성 만족 안 함 ...
from  scipy.stats import shapiro
print(shapiro(type_ya_df))
print(shapiro(type_yb_df))
print(shapiro(type_yc))
print(shapiro(type_yd))
print(shapiro(type_ye))

AttributeError: 'DataFrame' object has no attribute 'type_y'

In [8]:
#등분산성 확인, 등분산성도 만족 안 함
from scipy.stats import levene
print(levene(type_ya,type_yb,type_yc,type_yd,type_ye))

LeveneResult(statistic=219453.30893019, pvalue=0.0)


In [11]:
import pingouin as pg
#등분산 만족안해서 welch_anova로
pg.welch_anova(dv = 'type_y', between = 'treatment', data = df)
#평균값에 차이 있다

,Source,ddof1,ddof2,F,p-unc,np2
0,treatment,4,981789.338756,12243.441298,0.0,0.023083


In [12]:
#사후분석
pg.pairwise_gameshowell(dv = 'type_y', between = 'treatment', data = df)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,A,B,708.378165,328.275233,380.102932,3.043528,124.888909,7.854413e+05,0.001,0.255164
1,A,C,708.378165,197.790647,510.587518,2.737560,186.511933,5.671442e+05,0.001,0.330661
2,A,D,708.378165,352.084510,356.293655,2.831029,125.853075,6.444647e+05,0.001,0.216036
3,A,E,708.378165,270.285490,438.092675,3.110890,140.825520,7.296443e+05,0.001,0.355827
4,B,C,328.275233,197.790647,130.484586,1.603345,81.382705,6.238141e+05,0.001,0.149811
5,B,D,328.275233,352.084510,-23.809277,1.758183,-13.541979,8.502681e+05,0.001,-0.024195
6,B,E,328.275233,270.285490,57.989743,2.180394,26.595988,5.661101e+05,0.001,0.068481
7,C,D,197.790647,352.084510,-154.293863,1.149946,-134.174933,1.708684e+06,0.001,-0.197235
8,C,E,197.790647,270.285490,-72.494843,1.727798,-41.957931,2.999683e+05,0.001,-0.099278
9,D,E,352.084510,270.285490,81.799020,1.872372,43.687369,4.049040e+05,0.001,0.101540


# type_x 분산분석

In [30]:
type_x_Additional = train_merge_data['type_x'] == 'Additional'
type_x_Additional_df = train_merge_data.loc[type_x_Additional, :]
type_x_Additional_df = type_x_Additional_df[['sales']]
type_x_Additional_df.columns = ['type_x']
type_x_Additional_df['treatment'] = 'Additional'

type_x_Bridge = train_merge_data['type_x'] == 'Bridge'
type_x_Bridge_df = train_merge_data.loc[type_x_Bridge, :]
type_x_Bridge_df = type_x_Bridge_df[['sales']]
type_x_Bridge_df.columns = ['type_x']
type_x_Bridge_df['treatment'] ='Bridge'

type_x_Empty = train_merge_data['type_x'] == 'Empty'
type_x_Empty_df = train_merge_data.loc[type_x_Empty, :]
type_x_Empty_df = type_x_Empty_df[['sales']]
type_x_Empty_df.columns = ['type_x']
type_x_Empty_df['treatment'] ='Empty'

type_x_Event = train_merge_data['type_x'] == 'Event'
type_x_Event_df = train_merge_data.loc[type_x_Event, :]
type_x_Event_df = type_x_Event_df[['sales']]
type_x_Event_df.columns = ['type_x']
type_x_Event_df['treatment'] ='Event'

type_x_Holiday = train_merge_data['type_x'] == 'Holiday'
type_x_Holiday_df = train_merge_data.loc[type_x_Holiday, :]
type_x_Holiday_df = type_x_Holiday_df[['sales']]
type_x_Holiday_df.columns = ['type_x']
type_x_Holiday_df['treatment'] ='Holiday'

type_x_Transfer = train_merge_data['type_x'] == 'Transfer'
type_x_Transfer_df = train_merge_data.loc[type_x_Transfer, :]
type_x_Transfer_df = type_x_Transfer_df[['sales']]
type_x_Transfer_df.columns = ['type_x']
type_x_Transfer_df['treatment'] ='Transfer'

type_x_Work_Day = train_merge_data['type_x'] == 'Work Day'
type_x_Work_Day_df = train_merge_data.loc[type_x_Work_Day, :]
type_x_Work_Day_df = type_x_Work_Day_df[['sales']]
type_x_Work_Day_df.columns = ['type_x']
type_x_Work_Day_df['treatment'] ='Work Day'

df = pd.concat([type_x_Additional_df,type_x_Bridge_df,type_x_Empty_df,type_x_Event_df,type_x_Holiday_df,type_x_Transfer_df,type_x_Work_Day_df])
df

,type_x,treatment
231660,4.000,Additional
231661,0.000,Additional
231662,4.000,Additional
231663,1114.000,Additional
231664,0.000,Additional
...,...,...
2555379,529.684,Work Day
2555381,158.774,Work Day
2555383,1618.001,Work Day
2555385,3.000,Work Day


In [24]:
#등분산성, 등분산성 만족 못함
from scipy.stats import levene
#print(levene(df.type_x[df.treatment=='Additional'],
#      df.type_x[df.treatment=='Bridge'],
#      df.type_x[df.treatment=='Empty'],
#      df.type_x[df.treatment=='Holiday'],
#      df.type_x[df.treatment=='Transfer'],
#      df.type_x[df.treatment=='Work Day']))

#pg.welch_anova(dv = '양적자료', between = '집단(질적자료)', data = )
pg.homoscedasticity(dv = 'type_x', group = 'treatment', data = df)

,W,pval,equal_var
levene,254.657481,0.0,False


In [22]:
import pingouin as pg
#등분산 만족안해서 welch_anova로
pg.welch_anova(dv = 'type_x', between = 'treatment', data = df)
#평균값에 차이 있다

,Source,ddof1,ddof2,F,p-unc,np2
0,treatment,6,41348.080949,166.647097,3.465523e-210,0.000512


In [25]:
#사후분석
#등분산가정을 만족하지 않기 때문에 Games Howell로 검정
pg.pairwise_gameshowell(dv = 'type_x', between = 'treatment', data = df)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,Additional,Bridge,487.628925,446.750854,40.878071,18.719101,2.183762,6370.096612,0.304461,0.031023
1,Additional,Empty,487.628925,352.159181,135.469745,5.476413,24.736949,69811.121177,0.001000,0.096314
2,Additional,Event,487.628925,425.660065,61.968860,7.014748,8.834082,144459.053139,0.001000,0.043983
3,Additional,Holiday,487.628925,358.433812,129.195114,5.822477,22.189030,88771.226251,0.001000,0.094269
4,Additional,Transfer,487.628925,467.753110,19.875815,12.128060,1.638829,24738.813024,0.635849,0.014392
5,Additional,Work Day,487.628925,372.157524,115.471401,13.510594,8.546730,12619.035279,0.001000,0.096316
6,Bridge,Empty,446.750854,352.159181,94.591673,17.925435,5.276953,5360.128588,0.001000,0.072248
7,Bridge,Event,446.750854,425.660065,21.090789,18.453639,1.142907,6019.177342,0.900000,0.016044
8,Bridge,Holiday,446.750854,358.433812,88.317042,18.034172,4.897205,5491.358737,0.001000,0.067563
9,Bridge,Transfer,446.750854,467.753110,-21.002256,20.938480,-1.003046,9551.494844,0.900000,-0.015840


# locale 분산분석

In [26]:
locale_Local = train_merge_data['locale'] == 'Local'
locale_Local_df = train_merge_data.loc[locale_Local, :]
locale_Local_df = locale_Local_df[['sales']]
locale_Local_df.columns = ['locale']
locale_Local_df['treatment'] = 'Local'

locale_National = train_merge_data['locale'] == 'National'
locale_National_df = train_merge_data.loc[locale_National, :]
locale_National_df = locale_National_df[['sales']]
locale_National_df.columns = ['locale']
locale_National_df['treatment'] = 'National'

locale_Regional = train_merge_data['locale'] == 'Regional'
locale_Regional_df = train_merge_data.loc[locale_Regional, :]
locale_Regional_df = locale_Regional_df[['sales']]
locale_Regional_df.columns = ['locale']
locale_Regional_df['treatment'] = 'Regional'

locale_Empty = train_merge_data['locale'] == 'Empty'
locale_Empty_df = train_merge_data.loc[locale_Empty, :]
locale_Empty_df = locale_Empty_df[['sales']]
locale_Empty_df.columns = ['locale']
locale_Empty_df['treatment'] = 'Empty'

df = pd.concat([locale_Local_df,locale_National_df,locale_Regional_df,locale_Empty_df])


In [27]:
#등분산성, 만족 못함
pg.homoscedasticity(dv = 'locale', group = 'treatment', data = df)

,W,pval,equal_var
levene,277.716952,2.958176e-180,False


In [28]:
#검정, 평균값에 차이 있따
pg.welch_anova(dv = 'locale', between = 'treatment', data = df)

,Source,ddof1,ddof2,F,p-unc,np2
0,treatment,3,130645.303642,203.938689,5.634923e-132,0.000275


In [29]:
#사후분석
pg.pairwise_gameshowell(dv = 'locale', between = 'treatment', data = df)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,Empty,Local,352.159181,366.165397,-14.006216,2.607948,-5.370589,239267.813042,0.001000,-0.012233
1,Empty,National,352.159181,416.751069,-64.591889,2.685866,-24.048809,298162.776764,0.001000,-0.049340
2,Empty,Regional,352.159181,387.005786,-34.846605,6.945033,-5.017486,32687.102633,0.001000,-0.028191
3,Local,National,366.165397,416.751069,-50.585672,3.620459,-13.972172,467250.062582,0.001000,-0.041007
4,Local,Regional,366.165397,387.005786,-20.840389,7.357128,-2.832680,41052.082819,0.023893,-0.016989
5,National,Regional,416.751069,387.005786,29.745283,7.385108,4.027739,41691.137928,0.001000,0.023826
